In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [5]:
options = Options()
# options.add_argument('--headless')  # 필요 시 주석 해제
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # URL 열기
    url = 'https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx'
    driver.get(url)

    # 드롭다운 메뉴에서 삼성 선택하기
    wait = WebDriverWait(driver, 10)
    
    # 팀 선택 드롭다운 메뉴 찾기
    team_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')))
    select = Select(team_dropdown)
    
    # 삼성 옵션 선택하기
    # KIA = 'HT' 삼성 = 'SS" LG = 'LG' 두산 = 'OB' KT = 'KT' SSG = 'SK' 한화 = 'HH' 롯데 = 'LT' NC = 'NC' 키움 = 'WO'
    select.select_by_value('SS')

    # 선택 후 페이지 갱신 대기
    time.sleep(5)  # 페이지 로드 대기 시간 (조정 가능)
    
    # 페이지 소스 가져오기
    html = driver.page_source

    # BeautifulSoup으로 파싱하기
    soup = BeautifulSoup(html, 'html.parser')
   
finally:
    # 웹 드라이버 종료
    driver.quit()

table = soup.find('table', class_="tData01 tt")
thead = table.thead

head_list=[]
for col in thead.find_all('th'):
    head_list.append(col.text)

tbody = soup.find('tbody')
data = []
rank = 1

for tr in tbody.find_all('tr'):
    row = [rank]  # Add rank information

    # Collect cell data
    for col in tr.find_all('td'):
        row.append(col.get_text(strip=True))

    data.append(row)
    rank += 1

# Remove the rank column from each row
data = [row[1:] for row in data]

df = pd.DataFrame(data=data, columns=head_list)
column_mapping = {
    'ERA': '평균자책점',
    'G': '경기',
    'W': '승리',
    'L': '패배',
    'SV': '세이브',
    'HLD': '홀드',
    'WPCT': '승률',
    'IP': '이닝',
    'H': '피안타',
    'HR': '피홈런',
    'BB': '볼넷',
    'HBP': '사구',
    'SO' : '삼진',
    'R' : '실점',
    'ER' : '자책점',
    'WHIP' : '이닝당 출루허용률'
}

df.rename(columns=column_mapping, inplace=True)
df.set_index('순위', inplace=True)
df

,선수명,팀명,평균자책점,경기,승리,패배,세이브,홀드,승률,이닝,피안타,피홈런,볼넷,사구,삼진,실점,자책점,이닝당 출루허용률
순위,,,,,,,,,,,,,,,,,,
1,최지광,삼성,2.08,28,3,1,0,7,0.750,30 1/3,18,0,20,3,29,7,7,1.25
2,김태훈,삼성,3.32,43,3,2,0,17,0.600,40 2/3,36,4,15,3,28,16,15,1.25
3,황동재,삼성,3.51,11,0,1,0,0,0.000,25 2/3,25,1,7,2,21,11,10,1.25
4,원태인,삼성,3.52,24,13,6,0,0,0.684,138,121,16,36,6,103,57,54,1.14
5,이상민,삼성,3.68,26,1,1,1,3,0.500,22,23,0,4,3,9,9,9,1.23
6,레예스,삼성,3.75,23,9,4,0,0,0.692,127 1/3,144,13,27,4,100,57,53,1.34
7,코너,삼성,3.81,25,9,6,0,0,0.600,141 2/3,126,21,38,16,137,66,60,1.16
8,임창민,삼성,3.86,50,1,0,1,25,1.000,44 1/3,47,3,26,1,41,21,19,1.65
9,김재윤,삼성,3.99,57,4,8,5,25,0.333,58 2/3,49,11,25,1,47,33,26,1.26
